In [3]:
import random
import numpy as np
import pandas as pd
from scipy import stats
import pycaret
import gower
# import umap
import umap.umap_ as umap
from sklearn.neighbors import DistanceMetric
from pycaret.clustering import *
from category_encoders.backward_difference import BackwardDifferenceEncoder
from transformers import RobertaForMaskedLM, RobertaTokenizer, pipeline
from category_encoders import HelmertEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import gridspec
import matplotlib.pyplot as plt
import plotnine as p9
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [1]:
from sklearn_extra.cluster import KMedoids

ImportError: dlopen(/Users/anyarulina/miniconda3/envs/main/lib/python3.9/site-packages/sklearn_extra/utils/_cyfht.cpython-39-darwin.so, 0x0002): tried: '/Users/anyarulina/miniconda3/envs/main/lib/python3.9/site-packages/sklearn_extra/utils/_cyfht.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/anyarulina/miniconda3/envs/main/lib/python3.9/site-packages/sklearn_extra/utils/_cyfht.cpython-39-darwin.so' (no such file), '/Users/anyarulina/miniconda3/envs/main/lib/python3.9/site-packages/sklearn_extra/utils/_cyfht.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))

In [ ]:
dataset_transformed = pd.read_csv('embeddings.csv')

In [11]:
num_clusters = 50
random_state = 32

In [18]:
dataset.sample()

,REQUEST_SIZE,RESPONSE_CODE,bad_resp_code,bad_req_size,CLIENT_USERAGENT,MATCHED_VARIABLE_SRC,MATCHED_VARIABLE_NAME,MATCHED_VARIABLE_VALUE
35184,1425,404,0,0,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,REQUEST_COOKIES,REQUEST_COOKIES.User-cn,Ангелина Фролова


### Dimensionality Reduction with UMAP

In [19]:
# use Umap to do embedding then cluster on that
def plot_2d_umap(df, intersection=False):

  numerical = df.select_dtypes(exclude='object')

  for c in numerical.columns:
      numerical[c] = (numerical[c] - numerical[c].mean())/numerical[c].std(ddof=0)
      
  ##preprocessing categorical
  categorical = df.select_dtypes(include='object')
  categorical = pd.get_dummies(categorical)

  #Percentage of columns which are categorical is used as weight parameter in embeddings later
  categorical_weight = len(df.select_dtypes(include='object').columns) / df.shape[1]

  #Embedding numerical & categorical
  fit1 = umap.UMAP(random_state=12).fit(numerical)
  fit2 = umap.UMAP(metric='dice', n_neighbors=250).fit(categorical)

  numeric_embedding = fit1.embedding_
  numeric = pd.DataFrame(
                         {'x': numeric_embedding[:,0],
                         'y':  numeric_embedding[:,1],
                        })
  

  categorical_embedding = fit2.embedding_
  categorical = pd.DataFrame(
                         {'x': categorical_embedding[:,0],
                         'y':  categorical_embedding[:,1],
                        })


  # intersection will resemble the numerical embedding more.
  if intersection:
    embedding = fit1 * fit2

  # union will resemble the categorical embedding more.
  else:
    embedding = fit1 + fit2

  umap_embedding = embedding.embedding_


  results = pd.DataFrame(
                        {'x': umap_embedding[:,0],
                         'y':  umap_embedding[:,1],
                        })
  


  numeric = (p9.ggplot(numeric, p9.aes(x='x', y='y'))
              + p9.geom_point(alpha=0.6)
              + p9.ggtitle('Numeric Embedding'))
  
  categorical = (p9.ggplot(categorical, p9.aes(x='x', y='y'))
              + p9.geom_point(alpha=0.6)
              + p9.ggtitle('Categorical Embedding'))
  
  combined = (p9.ggplot(results, p9.aes(x='x', y='y'))
              + p9.geom_point(alpha=0.6)
              + p9.ggtitle('Combined Embedding'))
  
  return numeric, categorical, combined, umap_embedding

In [20]:
UMAP_2D_data = dataset.copy()

In [21]:
%%time
numeric, categorical, combined, embedding_ =  plot_2d_umap(UMAP_2D_data, intersection=True)

UMAP_2D_data['x'] = embedding_[:,0]
UMAP_2D_data['y'] = embedding_[:,1]

# UMAP_2D_data['target'] = labels


UMAP_clusterer = KMeans(n_clusters=num_clusters, init='k-means++',
                        verbose=0, random_state=random_state,
                        algorithm='auto').fit(embedding_)

UMAP_2D_data["preds"] = UMAP_clusterer.labels_
UMAP_2D_data["preds"] = UMAP_2D_data["preds"].apply(str)

predictions['UMAP_2D_preds'] = UMAP_clusterer.labels_

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


NameError: name 'KMeans' is not defined

In [22]:
p1_umap = (p9.ggplot(UMAP_2D_data, p9.aes(x='x', y='y', fill='target', color='target'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data UMAP Ground Truth')
      + p9.theme_bw())

p2_umap = (p9.ggplot(UMAP_2D_data, p9.aes(x='x', y='y', fill='preds', color='preds'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data UMAP Kmeans')
      + p9.theme_bw())

p1_umap

PlotnineError: "Could not evaluate the 'fill' mapping: 'target' (original error: name 'target' is not defined)"

In [23]:
p2_umap

PlotnineError: "Could not evaluate the 'fill' mapping: 'preds' (original error: name 'preds' is not defined)"

### Gower Distance

In [24]:
def calculate_gower_distance(df):
  
  '''
  Takes a dataframe as an input and returns the gower distance
  matrix.
  
  '''

  variable_distances = []
  
  for col in range(df.shape[1]):
    
    feature = df.iloc[:,[col]]
    if feature.dtypes.values == object:
      
      feature_dist = DistanceMetric.get_metric('dice').pairwise(pd.get_dummies(feature, drop_first=True))
        
    else:
      
      feature_dist = DistanceMetric.get_metric('manhattan').pairwise(feature) / max(np.ptp(feature.values),1)


      variable_distances.append(feature_dist)
      

  return np.array(variable_distances).mean(0)

In [ ]:
%%time 
gower_data = dataset.copy()
gower_distance_matrix = calculate_gower_distance(gower_data)

In [ ]:
umap_embeddings = umap.UMAP(random_state=42, n_components=2).fit_transform(gower_distance_matrix)

gower_data['x'] = umap_embeddings[:,0]
gower_data['y'] = umap_embeddings[:,1]

# gower_data['target'] = labels


gower_clusterer = KMedoids(n_clusters=num_clusters, metric='precomputed',
                           method='pam', init='build', 
                           max_iter=300, random_state=random_state).fit(gower_distance_matrix)


gower_data["preds"] = gower_clusterer.labels_
gower_data["preds"] = gower_data["preds"].apply(str)

predictions['gower_preds'] = gower_clusterer.labels_

In [ ]:
p1_gower_d = (p9.ggplot(gower_data, p9.aes(x='x', y='y', fill='target', color='target'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data Gower Matrix UMAP 2D Ground Truth')
      + p9.theme_bw())

p2_gower_d = (p9.ggplot(gower_data, p9.aes(x='x', y='y', fill='preds', color='preds'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data Gower Matrix UMAP 2D KMedoids')
      + p9.theme_bw())

p1_gower_d

In [ ]:
p2_gower_d

### FAMD

In [ ]:
def calculate_zscore(df, columns):
  '''
  scales columns in dataframe using z-score
  '''
  df = df.copy()
  for col in columns:
      df[col] = (df[col] - df[col].mean())/df[col].std(ddof=0)

  return df



def one_hot_encode(df, columns):
  '''
  one hot encodes list of columns and
  concatenates them to the original df
  '''

  concat_df = pd.concat([pd.get_dummies(df[col], drop_first=True, prefix=col) for col in columns], axis=1)
  one_hot_cols = concat_df.columns

  return concat_df, one_hot_cols



def normalize_column_modality(df, columns):
  '''
  divides each column by the probability μₘ of the modality 
  (number of ones in the column divided by N) only for one hot columns
  '''

  length = len(df)
  for col in columns:
    
    weight = math.sqrt(sum(df[col])/length)
    df[col] = df[col]/weight

  return df



def center_columns(df, columns):
  '''
  center columns by subtracting the mean value
  '''
  for col in columns:
      df[col] = (df[col] - df[col].mean())

  return df



def FAMD_(df, n_components=2):
  '''
  Factorial Analysis of Mixed Data (FAMD), 
  which generalizes the Principal Component Analysis (PCA) 
  algorithm to datasets containing numerical and categorical variables

  a) For the numerical variables
    - Standard scale (= get the z-score)
  
  b) For the categorical variables:
    - Get the one-hot encoded columns
    - Divide each column by the square root of its probability sqrt(μₘ)
    - Center the columns

  c) Apply a PCA algorithm over the table obtained!

  '''
  
  variable_distances = []

  numeric_cols = data.select_dtypes(include=np.number)
  cat_cols = data.select_dtypes(include='object')
  
  # numeric process
  normalized_df = calculate_zscore(df, numeric_cols)
  normalized_df = normalized_df[numeric_cols.columns]

  # categorical process
  cat_one_hot_df, one_hot_cols = one_hot_encode(df, cat_cols)
  cat_one_hot_norm_df = normalize_column_modality(cat_one_hot_df, one_hot_cols)
  cat_one_hot_norm_center_df = center_columns(cat_one_hot_norm_df, one_hot_cols)

  # Merge DataFrames
  processed_df = pd.concat([normalized_df, cat_one_hot_norm_center_df], axis=1)

  # Perform (PCA)
  pca = PCA(n_components=n_components)
  principalComponents = pca.fit_transform(processed_df)

  return principalComponents

In [ ]:
FAMD_data = dataset.copy()
FAMD_components = FAMD_(FAMD_data)

FAMD_data['x'] = FAMD_components[:,0]
FAMD_data['y'] = FAMD_components[:,1]

FAMD_data['target'] = labels


FAMD_clusterer = KMeans(n_clusters=num_clusters, init='k-means++',
                        verbose=0, random_state=random_state,
                        algorithm='auto').fit(FAMD_components)


FAMD_data["preds"] = FAMD_clusterer.labels_
FAMD_data["preds"] = FAMD_data["preds"].apply(str)

predictions['FAMD_preds'] = FAMD_clusterer.labels_

In [ ]:
p1_famd = (p9.ggplot(FAMD_data, p9.aes(x='x', y='y', fill='target', color='target'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data FAMD Ground Truth')
      + p9.theme_bw())

p2_famd = (p9.ggplot(FAMD_data, p9.aes(x='x', y='y', fill='preds', color='preds'))
      + p9.geom_point(alpha=0.8, size=2)
      + p9.ggtitle('Bank Data FAMD Kmeans')
      + p9.theme_bw())

p1_famd

In [ ]:
p2_famd

In [33]:
# пока без теста
# X_train, X_test, y_train, y_test = train_test_split(classified, y, test_size=0.2, random_state=42)

In [34]:
# %%time
# bd = BackwardDifferenceEncoder(drop_invariant=True)
# dataset_bd = bd.fit_transform(dataset[cat_features])
# dataset_f = pd.merge(dataset_bd, dataset[num_features], left_index=True, right_index=True)

In [ ]:
# %%time
# he = HelmertEncoder(drop_invariant=True)
# dataset_he = he.fit_transform(dataset[cat_features])
# dataset_f = pd.merge(dataset_he, dataset[num_features], left_index=True, right_index=True)

In [ ]:
# https://www.kaggle.com/code/karnikakapoor/customer-segmentation-clustering

In [ ]:
# https://github.com/art-glazunov/Non-standard-categorical-encodings/blob/main/group_counts_and_SVD.ipynb

In [8]:
# брать только часть датасета 
df_sample = dataset_transformed.sample(frac=0.25)
dataset_transformed.shape, df_sample.shape

((57845, 41), (14461, 41))

### Goverce distance 

In [ ]:
# https://medium.com/analytics-vidhya/the-ultimate-guide-for-clustering-mixed-data-1eefa0b4743b

In [ ]:
## очень долго считается

%%time
dist_matrix = gower.gower_matrix(df_sample)

In [ ]:


clusters = (KMedoids(n_clusters=num_clusters, 
                           metric='precomputed',
                           method='pam', init='build', 
                           max_iter=300, 
                           random_state=random_state)
            .fit(dist_matrix)
            .labels_)

### pycaret 

In [5]:
data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (54953, 8)
Unseen Data For Predictions: (2892, 8)


In [7]:
exp_clu101 = setup(data, normalize = True, 
                   # ignore_features = ['MouseID'],
                   session_id = 123)

KeyboardInterrupt: 

In [ ]:
# 'dbscan'
kmeans = create_model('kmeans')
print(kmeans)

In [ ]:
kmodes = create_model('kmodes', num_clusters = 6)
print(kmodes)

In [ ]:
models()

In [ ]:
kmean_results = assign_model(kmeans)
kmean_results.head()

In [ ]:
plot_model(kmeans)

In [ ]:
plot_model(kmeans, plot = 'elbow')

In [ ]:
plot_model(kmeans, plot = 'silhouette')

In [ ]:
plot_model(kmeans, plot = 'distribution', feature = 'class')

In [ ]:
unseen_predictions = predict_model(kmeans, data=data_unseen)
unseen_predictions.head()

In [ ]:
save_model(kmeans,’Final KMeans Model 25Nov2020')

In [ ]:
# TODO: посмотреть на метрики - насколько хорошо обучилась модель 